# TODO
* Tokenize text
* Investigate more model

In [1]:
# import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [1]:
import gc
import sys

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import (
    LSTM,
    Activation,
    Bidirectional,
    Dense,
    Dropout,
    Input,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

2024-05-09 15:23:14.540558: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-09 15:23:14.582124: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 15:23:14.582162: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 15:23:14.582172: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-09 15:23:14.590485: I tensorflow/core/platform/cpu_feature_g

In [2]:
gpus = tf.config.list_physical_devices("GPU")
print(gpus)
tf.config.set_logical_device_configuration(
    gpus[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=4000)],
)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-05-09 15:23:17.651768: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-09 15:23:17.669576: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-09 15:23:17.669796: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
text = open("./data/robert_frost.txt").read().lower()

In [4]:
chars = sorted(list(set(text)))

In [5]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [6]:
max_length = 55
step = 8

sentence = []
next_char = []

for i in range(0, len(text) - max_length, step):
    sentence.append(text[i : i + max_length])
    next_char.append(text[i + max_length])

sub_data = 1000
sentence = sentence[: len(sentence) // sub_data]
next_char = next_char[: len(next_char) // sub_data]

In [7]:
len(sentence), len(chars)

(14, 43)

In [8]:
X = np.zeros((len(sentence), max_length, len(chars)), dtype="int8")
Y = np.zeros((len(sentence), len(chars)), dtype="int8")

for i, sent in enumerate(sentence):
    for k, char in enumerate(sent):
        X[i, k, char_indices[char]] = 1
    Y[i, char_indices[next_char[i]]] = 1

In [9]:
X.shape

(14, 55, 43)

In [10]:
print(gc.collect())
K.clear_session()

model = Sequential()

layers = [
    Input(shape=(X.shape[1], X.shape[2])),
    # Bidirectional(LSTM(2048, return_sequences=True)),
    # Bidirectional(LSTM(2048, return_sequences=True)),
    # Bidirectional(LSTM(1024)),
    LSTM(1024),
    Dense(len(chars), activation="softmax"),
]

model = tf.keras.Sequential(layers)
model.compile(loss="categorical_crossentropy", optimizer="adam", run_eagerly=True)

43


2024-05-09 15:23:17.930593: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-09 15:23:17.930858: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-09 15:23:17.931002: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1024)              4374528   
                                                                 
 dense (Dense)               (None, 43)                44075     
                                                                 
Total params: 4418603 (16.86 MB)
Trainable params: 4418603 (16.86 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
es = keras.callbacks.EarlyStopping(
    monitor="loss", min_delta=0.01, patience=5, restore_best_weights=True
)

In [13]:
model.fit(X, Y, epochs=100, callbacks=[es])

Epoch 1/100


2024-05-09 15:23:20.337334: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8800
2024-05-09 15:23:20.678038: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5ac00f095810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-09 15:23:20.678060: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 with Max-Q Design, Compute Capability 7.5
2024-05-09 15:23:20.694453: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-09 15:23:20.844111: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 [==============================] - 1s 1s/step - loss: 3.7690
Epoch 2/100
1/1 [==============================] - 0s 65ms/step - loss: 3.6451
Epoch 3/100
1/1 [==============================] - 0s 63ms/step - loss: 3.3198
Epoch 4/100
1/1 [==============================] - 0s 54ms/step - loss: 3.5254
Epoch 5/100
1/1 [==============================] - 0s 67ms/step - loss: 2.9581
Epoch 6/100
1/1 [==============================] - 0s 52ms/step - loss: 3.2024
Epoch 7/100
1/1 [==============================] - 0s 43ms/step - loss: 3.0667
Epoch 8/100
1/1 [==============================] - 0s 57ms/step - loss: 2.7532
Epoch 9/100
1/1 [==============================] - 0s 55ms/step - loss: 2.4860
Epoch 10/100
1/1 [==============================] - 0s 57ms/step - loss: 2.3073
Epoch 11/100
1/1 [==============================] - 0s 40ms/step - loss: 2.3201
Epoch 12/100
1/1 [==============================] - 0s 44ms/step - loss: 2.4591
Epoch 13/100
1/1 [==============================] - 0s 42ms/ste

In [14]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas), preds


def genText(text_to_gen, number_gen=200, temp=1):
    generated = text_to_gen
    sent = text_to_gen
    for i in range(number_gen):
        x = np.zeros((1, max_length, len(chars)))
        for t, char in enumerate(sent):
            x[0, t, char_indices[char]] = 1

        pred = model.predict(x, verbose=0)[0]
        # next_index = np.argmax(pred,axis=1)
        next_index, _ = sample(pred, temp)
        next_char = indices_char[next_index]

        generated += next_char
        sent = sent[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()

In [15]:
text2gen = text[1000:1055]
genText(text2gen, 500)

hntwen;ne int;newv;a  
t i;  tihnanh


KeyboardInterrupt

